In [43]:
# Artificial Intellegence HW1 
# Jan 20, 2023
# Booth 32200
# Nic Marlton

### Psuedo Code
# dependencies
# dictionary of spot ids (pulled mannually from surfline.com)
# use spot ids list to query surfline api
    # note: either the conditions url or the wave url could be used
    # `conditions` returns ratings 'POOR' to 'EPIC', whereas `wave`
    # returns an optimal rating 0, 1, 2. I chose the optimal rating
    # given its relative simplicity
    # helpful overview of api below:
    # https://pkg.go.dev/github.com/mhelmetag/surflinef/v2#section-readme
# assign surfline ratings to photos
    # in a future itteration of this tool, I would want to pull photos
    # and ratings in real time. that would make this step unnecessary. 
    # for the purposes of this proof of concept, I am mannually assigning 
    # ratings. 
# sort 90% of photos into 4 X 4 - good v bad X crowded v quiet
# store remaining 10 % for test data

### Process after this point occurs in Google's Teachable Machine 

In [101]:
# Dependencies
import pandas as pd
import numpy as np
import requests
from random import randint


In [45]:
# Dictionary of spotIds 
spot_id_dict = {
    'PlesurePoint':'5842041f4e65fad6a7708807',
    'UpperTrestles':'5842041f4e65fad6a7708887',
    'SanClementeStateBeach':'5842041f4e65fad6a77088cf',
    'LowerTrestles':'5842041f4e65fad6a770888a',
    'SteamerLane':'5842041f4e65fad6a7708805',
    'TStreetOverview':'5842041f4e65fad6a7708830',
    'Church':'5842041f4e65fad6a770888b',
    'Strands':'5842041f4e65fad6a77088d5',
    'Jacks':'5842041f4e65fad6a770880b',
    'SaltCreekOverview':'5842041f4e65fad6a770882e',
    # 'Hightower':'584204214e65fad6a7709cbd', // excluding Hightower given it is in FL 
    'Lowers':'5842041f4e65fad6a770888a',
    # 'VenicePierSouthside':'5842041f4e65fad6a7708b17' // excluding Venice Pier given it is in FL
}

In [59]:
# Use spot_id_dict to query surfline api
def get_wave_data(spot_id):
    """
    Executes an API query to surfline.com for a given spotId, e.g.
    'PlesurePoint':'5842041f4e65fad6a7708807'. Specifically, it uses 
    the wave url

    Input:
        - spot_id from spot_id_dict

    Output:
        - 2 column df timestamp and optimalScore for given beach.
    """
    # API Call to sufrline
    login_base_url = 'https://services.surfline.com/trusted/token'
    base_url = 'https://services.surfline.com/kbyg/regions/forecasts/wave'
    response = requests.request("GET", base_url + '?spotId=' + spot_id + '&days=1')
    
    # Transform data for output in 
    df = pd.json_normalize(response.json()['data'],record_path =['wave'])
    df = df[['timestamp','surf.optimalScore']]
    return df.head(1)

def capture_current_conditions(spot_id_dict):
    """
    Captures wave data for a given day by running get_wave_data() 
    for each record in spot_id_dict. In a future version of this module
    it could also save records to a file or db for use in photo labeling

    Input: 
        - takes spot_id_dict as input.

    Output:
        - df of optimalScore and metadata for spots in spot_id_dict for a 
        given date
    """
    forecast_df = pd.DataFrame()
    for i in spot_id_dict:
        spot_forecast_df = get_wave_data(spot_id_dict[i])
        spot_forecast_df['spot'] = i
        spot_forecast_df['spot_id'] = spot_id_dict[i]
        spot_forecast_df['date'] = pd.Timestamp.today().date()
        forecast_df = pd.concat([forecast_df,spot_forecast_df])

    return forecast_df

forecast_df = capture_current_conditions(spot_id_dict)

In [74]:
# Save sample output for current date
forecast_df.to_csv('forcast_data.csv')

In [104]:
# Concatenate data to create list of files w & w/o surfers 
    # I did not include these csv files in the git repo because of their size.
    # they can be downlaoded with the photo data here:
    # https://universe.roboflow.com/surfline/surfer-spotting/dataset/1

def create_file_name_df():
    """ 
    """
    # Concatenate all file names from full Roboflow data set (~40k photos)
    df1 = pd.read_csv('test_annotations.csv') # csv describing Roboflow test data
    df2 = pd.read_csv('train_annotations.csv') # csv describing Roboflow train data 
    df3 = pd.read_csv('valid_annotations.csv') # csv describing Roboflow validation data
    annotations_df = pd.concat([df1,df2,df3],ignore_index=True)
    
    # Remove rows for non-CA beaches and NaN
    annotations_df['filename'] = annotations_df['filename'].apply(lambda x: np.nan 
            if str(x).__contains__('VenicePierSouthside') or str(x).__contains__('VenicePierSouthside') else x)
    df_mask = annotations_df['filename'].isna()
    annotations_df = annotations_df[~df_mask]
    
    # Get count of surfers in each file
    annotations_df = annotations_df.groupby(['filename']).agg({'filename':'count'})#drop_duplicates().reset_index()
    annotations_df = annotations_df.rename(columns={'filename':'count'}).reset_index()
    annotations_df['crowded'] = annotations_df['count'].apply(lambda x: 1 if x > 2 else 0)

    return annotations_df

# Dataframe of files that include surferss
files_w_surfers_df = create_file_name_df()

In [127]:
# Select 100 images with surfers and 100 images w/o surfers

def choose_200_files(files_w_surfers_df):
    """ 
    """
    to_encode_df = pd.DataFrame()
    
    # Select 100 images with surfers
        # We carry over count of surfers and bool indicating crowded/not
        # from files_w_surfers_df
        # note: 0 (few surfers: <= 2), or 1 (many surfers: > 2)
    for i in range (100):
        index = randint(0,len(files_w_surfers_df) - 1)
        frame = files_w_surfers_df.iloc[index].to_frame().T
        to_encode_df = pd.concat([to_encode_df,frame])
    
    # read in names of all files form .txt file
        # again this is not saved on GitHub, but can be created based on
        # file names in directories downloaded from Roboflow
    all_file_names_df = pd.read_csv('all_file_names.txt',sep="\s{4}",header=None,engine='python')
    all_file_names_df[['count','crowded']] = [0,0]
    all_file_names_df = all_file_names_df.rename(columns={0:'filename'})

    # Select 100 images without surfers
        # We pull a random file from all_file_names_df and search for it in
        # files_w_surfers_df. If we do not find it, we add to to_encode_df
    count = 0
    while count < 100:
        index = randint(0,len(all_file_names_df) - 1)
        file_path = all_file_names_df.iloc[index].to_frame().T
        search_val = str(file_path.iloc[0,0])
        if files_w_surfers_df.filename[files_w_surfers_df.filename == search_val].count() > 0:
            count = count
        else:
            to_encode_df = pd.concat([to_encode_df,file_path])
            count += 1

    return to_encode_df

to_encode_df = choose_200_files(files_w_surfers_df)



(200, 3)

In [ ]:
# Mannually code images as 0 (poor surf), 2 (good surf) 
    # note: we are ignoring the 1 optimalScore rating available on surfline
    # in order to simplify our classification